### **SEXY PANDAS**
#### An attempt to restyle Pandas Data Frames to reflect Excel-style formatting, and to make re-use of the functions easy
----

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns

In [8]:
import dataframe_image # use this package to help print out our new styles

#### **GLOBAL VARIABLES**

In [9]:
# set of predefined RGB colors
PALETTES_RGB = {
    'yellow':[252,239,166],
    'green':[122,188,129],
    'red':[231,114,112],
    'white':[255,255,255],
    'blue':[101,147,194],
    'grey':[144,144,148],
    'sns_blue':[13,29,85],
    'sns_yellow':[255,255,221],
    'sns_green':[103,182,193]
}
PALETTES_HEX = {}

#### **HELPER FUNCTIONS**

In [10]:
# extract the hex value from a given color and round accordingly, ensuring length==2
def make_hex_color(s, round='nearest'):
    if round=='up':
        s_round = np.ceil(s,0)
    elif round=='down':
        s_round = np.floor(s,0)
    else:
        s_round = np.round(s,0)
    return ('0'+hex(int(s_round))[2:].upper())[-2:]

In [11]:
# make a full hex color from 3 RGB channels
def rgb_to_hex(channels, round='nearest'):
    return '#'+(''.join([make_hex_color(c, round) for c in channels]))

In [12]:
# use our helpers to populate hex code dict (we will want to speak generally in hex for this work)
for i in PALETTES_RGB.keys():
    PALETTES_HEX[i] = '#'+(''.join([make_hex_color(color) for color in PALETTES_RGB[i]]))

In [13]:
# make a range of evenly spaced floats of a given min, max and length
def divide_range(mymin, mymax, size):
    return [mymin+(k*(mymax-mymin)/(size-1)) for k in range(size)]
    # np.arange(mymin,mymax+(1/(size-1)),(1/(size-1))) # alternative way

In [14]:
def make_quantiles(values, n, mn, mx, spacing='relative'):
    if type(n)==list: n=len(n)
    if spacing == 'even':  # evenly distribute the color palette ignoring the magnitude of the values
        return [np.floor((n-1)*((values<=v).mean()+(values<v).mean())/2) for v in values]
    elif spacing == 'relative':  # factor in the megnitude of the values when making the visuals (default)
        return [np.maximum(0,np.minimum(int((n-1)*(v-mn)/(mx-mn)),n-2)) for v in values] # prevent negative values

In [15]:
# get RGB colors from hex if we want to go the other way
def get_rgb_colors(c):
    if c in PALETTES_RGB:
        return PALETTES_RGB[c]
    else:
        c = c.replace('#','')
        n = [c[i*int(len(c)/3):(i+1)*int(len(c)/3)] for i in range(3)]
        if len(c)==3: n = [s+s for s in n]
        return [int(n,16) for n in n]

In [117]:
# generates an RGB color value from a given float, based on its distance from defined min/max values and their associated rgb colors
def generate_color(value, thresholds, colors):
    (min,max) = thresholds
    (min_color,max_color) = colors
    diff = [min_color[i]-max_color[i] for i in range(3)]
    return [min_color[j]-(diff[j]*(value-min)/(max-min)) for j in range(3)]

In [17]:
def luminosity(v):
    return (0.2126*v[0]+0.7152*v[1]+0.0722*v[2])

#### **MAIN STYLING FUNCTIONS**

In [18]:
def apply_colors(col, palette=['yellow', 'green'], default_fill_color='#FFF', default_text_color='#000', type='shade', rows=None, columns=None, mymin=None, mymax=None):
    # by default, use column-wise min and max if nothing is provided
    if mymax is None: mymin, mymax = min(col.values), max(col.values)
    
    # to prevent a divide by zero later on - max must always be greater than min
    if mymax==mymin: mymax=mymin+1
    palette = [get_rgb_colors(p) for p in palette]
    
    if len(palette) == 1:
        # if the palette length is just 1 we just apply it globally - the trivial case
        rgb_vals = [palette[0] for c in col.values]
    else:
        # if the palette length is greater than 1, we assign each value a bespoke color based on its position in the full range
        thresholds = divide_range(mymin, mymax, len(palette))
        quantiles = make_quantiles(col.values, palette, mymin, mymax)
        rgb_vals = [generate_color(c, thresholds[q:q+2], palette[q:q+2]) for c,q in zip(col.values, quantiles)]

    def filter_cells(inputs, default=''):  
        if (columns is not None):
            inputs = [inputs[j] if (mymin <= col.values[j] <= mymax) and (col.name in columns) else default for j in range(len(col.values))]
        if (rows is not None):
            inputs = [inputs[j] if (mymin <= col.values[j] <= mymax) and (j in rows) else default for j in range(len(col.values))]
        return inputs
    
    if type == 'shade':
        res = ['background-color: #'+(''.join([make_hex_color(c) for c in v])) for v in rgb_vals]
        default = 'background-color: '+default_fill_color
        return filter_cells(res, default)
    elif type == 'text_shade':
        tx = ['color: '+('#000' if luminosity(v)>=100 else '#FFF') for v in rgb_vals]
        default = 'color: '+default_text_color
        return filter_cells(tx, default)
    else:
        return ['' for c in col.values]

#####
The **pretty_pandas** function acts as the main entry point to the styling, and can work with text and fill colors and other formatting:

In [19]:
def pretty_pandas(df, fill_palette=['yellow','green'], rows=None, columns=None, index='show', group=None, font_size=None, header_size=None,
                  default_fill_color='#FFF', default_text_color='#000', bg='white', mymin=None, mymax=None):
    """Generate efficient dataframe styling with fully customizable inputs.

    Keyword arguments:
    todo
    """
    sdf = df.style
    rows_all,columns_all = list(df.index),list(df.columns)
    if mymin is None: mymin=np.min(df.values)
    if mymax is None: mymax=np.max(df.values)

    if index=='hide': sdf.hide_index()
    if header_size is None: header_size=font_size
    if type(fill_palette[0]) != list: fill_palette=[fill_palette] 
    
    for palette in fill_palette:
        row_subset = rows_all if rows is None else [r for r in rows if r in rows_all]
        row_index_subset = [rows_all.index(r) for r in row_subset]
        col_subset = columns_all if columns is None else [c for c in columns if c in columns_all]
        d = df.loc[row_subset,col_subset]
        mymin = max(mymin, np.min(d.values)) if group is None else None
        mymax = min(mymax, np.max(d.values)) if group is None else None
        sdf.apply(apply_colors, palette=palette, default_fill_color=default_fill_color, default_text_color=default_text_color,
                  type='shade', rows=row_index_subset, columns=col_subset, mymin=mymin, mymax=mymax, axis=0)
        sdf.apply(apply_colors, palette=palette, default_fill_color=default_fill_color, default_text_color=default_text_color,
                  type='text_shade', rows=row_index_subset, columns=col_subset, mymin=mymin, mymax=mymax, axis=0)

    return sdf.format('{:.3f}').set_table_styles([{'selector':'tr','props':[('background-color',bg+' !important')]}])

    # sdf.set_properties(**{'font-size': str(font_size)+'pt'})
    # .set_table_styles([{'selector': 'th', 'props': [('font-size', str(22)+'pt !important')]}])

#### **EXAMPLES**

In [20]:
alpha = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
test_df = pd.DataFrame([np.arange(26)+(2*np.random.random(26)) for i in range(26)],columns=[a for a in alpha][:26])
# test_df = np.round(test_df,2)

fruits = ['Apple','Watermelon','Orange','Pear','Cherry','Strawberry','Nectarine','Grape','Mango','Blueberry','Pomegranate','Starfruit','Plum','Banana',
          'Raspberry','Mandarin','Jackfruit','Papaya','Kiwi','Pineapple','Lime','Lemon','Apricot','Grapefruit','Melon','Coconut','Avocado','Peach']

test_df.index = fruits[:26]
np.random.shuffle(fruits)

In [21]:
pretty_pandas(test_df)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


In [26]:
pretty_pandas(
    test_df, index='show', font_size=11, header_size=12, mymax=100,
    # fill_palette=['#FFFFDD','#DAECB8','#87C6BD','#4B96BE','#2E4C9B','#0D1D55'],
    fill_palette=['#e8f6b1', '#b2e1b6', '#65c3bf', '#2ca1c2', '#216daf', '#253997','#000'],
    rows = list(test_df.index)[8:18], #['Starfruit','Plum','Banana','Raspberry'],
    columns = ['B','C','D','E','F','G','H','I'],
    default_fill_color = '#F9F9F9',
    default_text_color = '#DDDDE4',
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


In [27]:
pretty_pandas(
    test_df, index='show', font_size=11, header_size=12, mymin=4, mymax=18,
    fill_palette=[list(sns.color_palette('YlOrRd').as_hex()),[]],
    default_fill_color = '#F9F9F9',
    default_text_color = '#555',
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


In [ ]:
#.hide_index()

#cols=[[['sns_yellow','sns_green','sns_blue'],'shade']],
#[['sns_blue','sns_green'],'shade',['A','C']]]

#).format('{:.3f}').hide_index()

#.highlight_max(color='lightgreen')
#.bar(subset='C',color='#AAC')

In [28]:
# USE CASES

# positive and negative values
# using logarithmic/exponential color scales
# finding outliers

In [376]:
def apply_colors(col, default_fill_color='#FFF', default_text_color='#000', default_fill_text_colors=['#000','#FFF'],
                 fill_palette=None, text_palette=None, rows=None, columns=None, mymin=None, mymax=None):

    fill_palette = [None if f is None else list(get_rgb_colors(p) for p in f) for f in fill_palette]
    text_palette = [None if t is None else list(get_rgb_colors(p) for p in t) for t in text_palette]
    rgb_fill_vals,rgb_text_vals,fill_styles,default_text_styles,active_text_styles = [],[],[],[],[]
    default = 'background-color: '+default_fill_color+'; color: '+default_text_color
    styles = [default for j in range(len(col.values))]
    text_styles = ['' for j in range(len(col.values))]
    
    for i in range(len(fill_palette)):
        if fill_palette[i] is not None:
            if len(fill_palette[i]) == 1: # if the palette length is just 1 we just apply it globally - the trivial case
                rgb_fill_vals += [[fill_palette[i][0] for c in col.values]]
            else: # if the palette length is greater than 1, we assign each value a bespoke color based on its position in the full range
                fill_thresholds = divide_range(mymin[i], mymax[i], len(fill_palette[i]))
                fill_quantiles = make_quantiles(col.values, fill_palette[i], mymin[i], mymax[i])
                rgb_fill_vals += [[generate_color(c, fill_thresholds[q:q+2], fill_palette[i][q:q+2]) for c,q in zip(col.values, fill_quantiles)]]
        else:
            rgb_fill_vals += [[None for c in col.values]]

        if text_palette[i] is not None:
            if len(text_palette[i]) == 1:
                rgb_text_vals += [[text_palette[i][0] for c in col.values]]
            else:
                text_thresholds = divide_range(mymin[i], mymax[i], len(text_palette[i]))
                text_quantiles = make_quantiles(col.values, text_palette[i], mymin[i], mymax[i])
                rgb_text_vals += [[generate_color(c, text_thresholds[q:q+2], text_palette[i][q:q+2]) for c,q in zip(col.values, text_quantiles)]]
        else:
            rgb_text_vals += [[None for c in col.values]]

        fill_styles += [['background-color: '+(default_fill_color if fill_palette[i] is None else '#'+''.join([make_hex_color(c) for c in v])) for v in rgb_fill_vals[-1]]]
        default_text_styles += [['color: '+(default_text_color if fill_palette[i] is None else (default_fill_text_colors[0] if luminosity(v)>=100 else default_fill_text_colors[1])) for v in rgb_fill_vals[-1]]]
        text_styles = [text_styles[j] if text_palette[i] is None else 'color: #'+''.join([make_hex_color(c) for c in rgb_text_vals[-1][j]]) for j in range(len(col.values))]
        styles = ['; '.join([fill_styles[i][j],text_styles[j]]) if (mymin[i] <= col.values[j] <= mymax[i]) and
                  (col.name in columns[i]) and (j in rows[i]) else styles[j] for j in range(len(col.values))]

    return styles

In [377]:
def pretty_pandas(df, fill_palette=None, text_palette=None, rows=None, columns=None, index='show', group=None, font_size=None, header_size=None,
                  default_fill_color='#FFF', default_text_color='#000', default_fill_text_colors=['#000','#FFF'], bg='white', mymin=None, mymax=None, configs=None):
    """Generate efficient dataframe styling with fully customizable inputs.
    Keyword arguments:
    
    todo
    """
    sdf = df.style
    rows_all,columns_all = list(df.index),list(df.columns)
    
    def absent():
        return None
    if configs is not None:
        configs_default = [defaultdict(absent) for c in configs]
        for c in range(len(configs)):
            for i in configs[c].keys():
                configs_default[c][i] = configs[c][i]
        fill_palette = [c['fill_palette'] for c in configs_default]
        text_palette = [c['text_palette'] for c in configs_default]
        rows = [c['rows'] if c['rows'] is not None else rows_all for c in configs_default]
        row_indices = [list(rows_all.index(i) for i in r) for r in rows]
        columns = [c['columns'] if c['columns'] is not None else columns_all for c in configs_default]
        mymin = [c['mymin'] if c['mymin'] is not None else np.min(df.loc[c['rows'],c['columns']].values) for c in configs_default]
        mymax = [c['mymax'] if c['mymax'] is not None else np.max(df.loc[c['rows'],c['columns']].values) for c in configs_default]
        mymax = [max(mymin[m]+1,mymax[m]) for m in range(len(mymax))] # to prevent any divide by zero later on
    else:
        if mymin is None: mymin=np.min(df.values)
        if mymax is None: mymax=np.max(df.values)
        if rows is None: rows = rows_all
        if columns is None: columns = columns_all
        mymin,mymax,rows,columns = [mymin],[mymax],[rows],[columns]
        
    if index=='hide': sdf.hide_index()
    if header_size is None: header_size=font_size

    sdf.apply(apply_colors, default_fill_color=default_fill_color, default_text_color=default_text_color, default_fill_text_colors=default_fill_text_colors,
              fill_palette=fill_palette, text_palette=text_palette, rows=row_indices, columns=columns, mymin=mymin, mymax=mymax, axis=0)

    return sdf.format('{:.3f}').set_table_styles([{'selector':'tr','props':[('background-color',bg+' !important')]}])

    # sdf.set_properties(**{'font-size': str(font_size)+'pt'})
    # .set_table_styles([{'selector': 'th', 'props': [('font-size', str(22)+'pt !important')]}])

In [379]:
configs=[
    {
        'mymin': 1,
        'mymax': 18,
        'fill_palette': list(sns.color_palette('YlOrRd').as_hex()),
        #'text_palette': ['#DDD','#500','#000'],
        'rows': list(test_df.index)[8:18],
        'columns': ['B','C','D','E','F','G','H','I','J','K','L','M']
    },
    {
        #'mymin': 20,
        #'mymax': 44,
        'fill_palette': ['#e8f6b1', '#b2e1b6', '#65c3bf', '#2ca1c2', '#216daf', '#253997', '#000'],
        #'text_palette': ['#AAF','#500','#000'],
        'rows': list(test_df.index)[2:11],
        'columns': ['L','M','N']
    }
]

pretty_pandas(
    test_df,
    index='show',
    font_size=11,
    header_size=12,
    default_fill_color='#F9F9FF',
    default_text_color='#555',
    configs=configs # this would be an alternative to passing values directly
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


In [357]:
from collections import defaultdict

def absent():
    return None
if configs is not None:
    configs_default = [defaultdict(absent) for c in configs]
    for c in range(len(configs)):
        for i in configs[c].keys():
            configs_default[c][i] = configs[c][i]
    mymin = [c['mymin'] for c in configs_default]
    mymax = [c['mymin'] for c in configs_default]
    fill_palette = [c['fill_palette'] for c in configs_default]
    rows = [c['rows'] for c in configs_default]
    columns = [c['columns'] for c in configs_default]

In [66]:
fill_palette

[['#ffea9b', '#fece6a', '#fea245', '#fc6832', '#ea2920', '#c20325'],
 ['#e8f6b1', '#b2e1b6', '#65c3bf', '#2ca1c2', '#216daf', '#253997', '#000']]

In [75]:
palette = [['#ffea9b', '#fece6a', '#fea245', '#fc6832', '#ea2920', '#c20325'],
           ['#e8f6b1', '#b2e1b6', '#65c3bf', '#2ca1c2', '#216daf', '#253997', '#000']]

In [96]:
z = [list(get_rgb_colors(p) for p in q) for q in palette]

In [98]:
z

[[[255, 234, 155],
  [254, 206, 106],
  [254, 162, 69],
  [252, 104, 50],
  [234, 41, 32],
  [194, 3, 37]],
 [[232, 246, 177],
  [178, 225, 182],
  [101, 195, 191],
  [44, 161, 194],
  [33, 109, 175],
  [37, 57, 151],
  [0, 0, 0]]]

In [86]:
range(len(palette))

range(0, 2)